In [67]:
import numpy as np
import pandas as pd
import time

In [68]:
def metric_df(df):
    truth = np.array(df['truth'])
    pred = np.array(df['pred'])
    diff = abs(pred - truth) / truth
#     print(list(diff <= 0.1).count(True) / len(diff))
    return list(diff <= 0.1).count(True) / len(diff)

In [69]:
test = pd.read_csv('dataset/test.csv')
train = pd.read_csv('dataset/train.csv')

valid_result_1 = pd.read_pickle('./output/xgb_area_kfold_valid_result.pkl')
submission = pd.read_csv('./output/submission_xgb_area_kfold_trainByAlltrain.csv')

valid_result_2 = pd.read_pickle('./output/lgbm_area_kfold_valid_result.pkl')
submission2 = pd.read_csv('./output/submission_lgbm_area_kfold_trainByAlltrain.csv')

In [70]:
def ensemble_by_previous_validate_group_accuracy(sub1, sub2, test, valid, cols, val_result_1, val_result_2):
    ### all arg. are series

    sub1_val_groupacc = val_result_1.groupby(cols).apply(metric_df) ###
#     print(sub1_val_groupacc)
    sub2_val_groupacc = val_result_2.groupby(cols).apply(metric_df) ###
    test_col_value = test.loc[:, cols]
    per_group_count = valid.groupby(cols).agg({'building_id': ['count']})
    
    sub1.sort_index(inplace=True)
    sub2.sort_index(inplace=True)
    test_col_value.sort_index(inplace=True) ###
    per_group_count.sort_index(inplace=True)
    subdf = pd.concat([sub1, sub2, test_col_value], axis=1) ###
    subdf['applymore_sub2'] = pd.Series()
    subdf['applymore_sub1'] = pd.Series()
    subdf['apply_avg'] = pd.Series()
    subdf['ensemble'] = pd.Series()
    
    sub1_val_groupacc.sort_index(inplace=True)
    sub2_val_groupacc.sort_index(inplace=True)
    valdf = pd.concat([sub1_val_groupacc, sub2_val_groupacc], axis=1)
#     print(valdf)
    
    print(sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] > valdf.iloc[:, 1])[0]))), \
          sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] < valdf.iloc[:, 1])[0]))), \
          sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] == valdf.iloc[:, 1])[0]))))
    
    val2_greater_group = list(np.where(valdf.iloc[:, 0] < valdf.iloc[:, 1])[0]) ###
    val1_greater_group = list(np.where(valdf.iloc[:, 0] > valdf.iloc[:, 1])[0]) ###
    
    subdf['applymore_sub2'] = subdf.iloc[:, 2].isin(val2_greater_group) ###
    subdf['applymore_sub1'] = subdf.iloc[:, 2].isin(val1_greater_group)
    subdf['apply_avg'] = ~subdf.iloc[:, 2].isin(val1_greater_group + val2_greater_group)
    
#     subdf['ensemble'] = subdf.iloc[:, 1] * subdf['apply_sub2'] * 0.5 + subdf.iloc[:, 0] * subdf['apply_sub2'] * 0.5 + \
#                         subdf.iloc[:, 0] * subdf['apply_sub1']
    subdf['ensemble'] = subdf.iloc[:, 1] * subdf['applymore_sub2'] * 0.55 + subdf.iloc[:, 0] * subdf['applymore_sub2'] * 0.45 + \
                        subdf.iloc[:, 0] * subdf['applymore_sub1'] * 0.55 + subdf.iloc[:, 1] * subdf['applymore_sub1'] * 0.45 + \
                        subdf.iloc[:, 0] * subdf['apply_avg'] * 0.5 + subdf.iloc[:, 1] * subdf['apply_avg'] * 0.5
            
#     print(subdf)
    return subdf['ensemble']



# per_town_count = train.groupby('town').agg({'building_id': ['count']})
ts = time.time()
result = ensemble_by_previous_validate_group_accuracy(submission['total_price'].reset_index(drop=True), 
                                                      submission2['total_price'], test, train, ['town'],
                                                      valid_result_1, valid_result_2)
print(time.time() - ts)
# result

2589 4337 1029
0.8177909851074219


In [71]:
submission3 = pd.DataFrame({
    "building_id": test.building_id, 
    "total_price": result
})
submission3.to_csv('./output/submission_xgb_area_lgbm_area_kfold_town_ensemble.csv', index=False)

In [72]:
submission3.head(10)

,building_id,total_price
0,X5gsdTWGS3W7JJQB,1.117064e+07
1,BTshNOJyKHnT2YIT,3.896438e+06
2,dhdymr0lV8N5kZOT,1.038430e+07
3,VEwyGGMcD56w5BOc,5.924031e+06
4,wmUeMoJZfsqaSX9b,1.056708e+06
5,EtBjGAHmHCe9t7TZ,3.024900e+06
6,hPNH34vmaZtvBtqc,1.225924e+07
7,wXjeI38bYDMJJwZC,6.507208e+06
8,fxZSGX6aPAFKU8W4,1.644733e+06
9,ewr0Fx6ign87OwaV,4.444172e+06


In [73]:
sum(submission3.total_price == submission2.total_price)

1

In [74]:
def ensemble_by_previous_validate_group_accuracy(sub1, sub2, test, valid, cols, val_result_1, val_result_2):
#     ### all arg. are series

#     sub1_val_groupacc = val_result_1.groupby(cols).apply(metric_df) ###
# #     print(sub1_val_groupacc)
#     sub2_val_groupacc = val_result_2.groupby(cols).apply(metric_df) ###
#     test_col_value = test.loc[:, cols]
#     per_group_count = valid.groupby(cols).agg({'building_id': ['count']})
    
#     sub1.sort_index(inplace=True)
#     sub2.sort_index(inplace=True)
#     test_col_value.sort_index(inplace=True) ###
#     per_group_count.sort_index(inplace=True)
#     subdf = pd.concat([sub1, sub2, test_col_value], axis=1) ###
#     subdf['apply_sub2'] = pd.Series()
#     subdf['apply_sub1'] = pd.Series()
#     subdf['ensemble'] = pd.Series()
    
#     sub1_val_groupacc.sort_index(inplace=True)
#     sub2_val_groupacc.sort_index(inplace=True)
#     valdf = pd.concat([sub1_val_groupacc, sub2_val_groupacc], axis=1)
# #     print(valdf)
    
#     print(sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] > valdf.iloc[:, 1])[0]))), \
#           sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] < valdf.iloc[:, 1])[0]))), \
#           sum(subdf.iloc[:, 2].isin(list(np.where(valdf.iloc[:, 0] == valdf.iloc[:, 1])[0]))))
    
#     val2_greater_group = list(np.where(valdf.iloc[:, 0] + 0.2 < valdf.iloc[:, 1])[0]) ###
#     ok_group = list(np.where(per_group_count >= 20)[0])
#     apply_sub2_group = list(set(val2_greater_group) & set(ok_group))
#     print(len(apply_sub2_group))
    
#     subdf['apply_sub2'] = subdf.iloc[:, 2].isin(apply_sub2_group) ###
#     subdf['apply_sub1'] = ~subdf['apply_sub2']
    
# #     subdf['ensemble'] = subdf.iloc[:, 1] * subdf['apply_sub2'] * 0.5 + subdf.iloc[:, 0] * subdf['apply_sub2'] * 0.5 + \
# #                         subdf.iloc[:, 0] * subdf['apply_sub1']
#     subdf['ensemble'] = subdf.iloc[:, 1] * subdf['apply_sub2']  + \
#                         subdf.iloc[:, 0] * subdf['apply_sub1']
            
# #     print(subdf)
#     return subdf['ensemble']



# # per_town_count = train.groupby('town').agg({'building_id': ['count']})
# ts = time.time()
# result = ensemble_by_previous_validate_group_accuracy(submission['total_price'].reset_index(drop=True), 
#                                                       submission2['total_price'], test, train, ['village'],
#                                                       valid_result_1, valid_result_2)
# print(time.time() - ts)
# # result

SyntaxError: unexpected EOF while parsing (<ipython-input-74-ae74a6fd289b>, line 52)

In [ ]:
# def ensemble_by_previous_validate_group_accuracy(sub1, sub2, test_col_value, per_town_count, sub1_val_groupacc, sub2_val_groupacc):
#     ### all arg. are series

#     sub1.sort_index(inplace=True)
#     sub2.sort_index(inplace=True)
#     test_col_value.sort_index(inplace=True)
#     per_town_count.sort_index(inplace=True)
#     subdf = pd.concat([sub1, sub2, test_col_value], axis=1)
#     subdf['apply_sub2'] = pd.Series()
#     subdf['apply_sub1'] = pd.Series()
#     subdf['ensemble'] = pd.Series()
    
#     sub1_val_groupacc.sort_index(inplace=True)
#     sub2_val_groupacc.sort_index(inplace=True)
#     valdf = pd.concat([sub1_val_groupacc, sub2_val_groupacc], axis=1)
    
#     val2_greater_town = list(np.where(valdf.iloc[:, 0] + 0.1 < valdf.iloc[:, 1])[0])
#     ok_town = list(np.where(per_town_count >= 20)[0])
#     apply_sub2_town = list(set(val2_greater_town) & set(ok_town))
    
#     subdf['apply_sub2'] = subdf.iloc[:, 2].isin(apply_sub2_town)
#     subdf['apply_sub1'] = ~subdf['apply_sub2']
    
#     subdf['ensemble'] = subdf.iloc[:, 1] * subdf['apply_sub2'] + subdf.iloc[:, 0] * subdf['apply_sub1']
            
#     print(subdf)
#     return subdf['ensemble']



# per_town_count = train.groupby('town').agg({'building_id': ['count']})
# ts = time.time()
# result = ensemble_by_previous_validate_group_accuracy(submission['total_price'].reset_index(drop=True), 
#                                                       submission2['total_price'], test['town'], per_town_count, 
#                                                       valid_result_town, valid_result_town2)
# print(time.time() - ts)

In [ ]:
# def ensemble_by_previous_validate_group_accuracy(sub1, sub2, test_col_value, per_town_count, sub1_val_groupacc, sub2_val_groupacc):
#     ### all arg. are series

#     sub1.sort_index(inplace=True)
#     sub2.sort_index(inplace=True)
#     test_col_value.sort_index(inplace=True)
#     per_town_count.sort_index(inplace=True)
#     subdf = pd.concat([sub1, sub2, test_col_value], axis=1)
#     subdf['ensemble'] = pd.Series()
    
#     sub1_val_groupacc.sort_index(inplace=True)
#     sub2_val_groupacc.sort_index(inplace=True)
#     valdf = pd.concat([sub1_val_groupacc, sub2_val_groupacc], axis=1)
    
#     val2_greater_ind = list(np.where(valdf.iloc[:, 0] + 0.1 < valdf.iloc[:, 1])[0])
#     ok_town = list(np.where(per_town_count >= 20)[0])
#     print(ok_town)
    
#     for i in range(len(subdf)):
#         if i % 1000 == 0:
#             print(i)
#         if subdf.iloc[i, 2] in val2_greater_ind and subdf.iloc[i, 2] in ok_town:
# #         if subdf.iloc[i, 2] in val2_greater_ind:
#             subdf.iloc[i, 3] = subdf.iloc[i, 1]
#         else:
#             subdf.iloc[i, 3] = subdf.iloc[i, 0]
            
#     print(subdf)
#     return subdf['ensemble']